Domain Question 1: How do temperature patterns in Chicago evolve during the winter-to-spring transition period?

Domain Question 2: How do environmental sensors in Chicago capture spatial variations in temperature?

Domain Question 3: What relationships exist between temperature, humidity, and precipitation in Chicago?

Domain Question 4: How does soil moisture respond to precipitation events in Chicago?

Domain Question 5: How do wind patterns vary across different parts of Chicago, and how do they correlate with temperature?

Domain Question 6: How do daily cycles of temperature and humidity change throughout the winter-to-spring transition?

In [1]:
# -*- coding: utf-8 -*-


# --- Block 1: Setup & Imports ---
# Import libraries
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt # Still useful for some settings or quick checks
import numpy as np
from datetime import datetime
from matplotlib.colors import LinearSegmentedColormap # Keep if needed for custom palettes
from shapely.geometry import Point
import matplotlib.dates as mdates # Keep if needed
import json # Needed for handling GeoJSON dict and saving specs
import urllib.parse # For encoding API queries
import os # For creating directories
import calendar # For month names

# Set plot style (affects matplotlib if used)
plt.style.use('ggplot')

# For better visualization (optional, mainly for matplotlib)
import matplotlib
matplotlib.rcParams['figure.figsize'] = (12, 8)

# Import Altair and enable rendering/fusion
import altair as alt
# Enable VegaFusion for potentially large datasets
try:
    alt.data_transformers.enable('default')
    print("default enabled.")
except ImportError:
    print("VegaFusion not installed, large datasets might be slow. Using default transformer.")
    alt.data_transformers.enable('default')

# For better rendering in Google Colab (if applicable)
# try:
#     alt.renderers.enable('colab')
#     print("Altair Colab renderer enabled.")
# except Exception as e:
#     print(f"Colab renderer not available: {e}. Using default renderer.")
#     alt.renderers.enable('default')

# Ensure necessary directories exist for saving data and specs
os.makedirs('data', exist_ok=True)
os.makedirs('specs', exist_ok=True)

# Global map for month numbers to names
month_names_map = {i: calendar.month_name[i] for i in range(1, 13)}


print("(Success) Block 1: Setup & Imports complete. Directories 'data' and 'specs' ensured.")

default enabled.
(Success) Block 1: Setup & Imports complete. Directories 'data' and 'specs' ensured.


In [2]:
# --- Block 2: Data Fetching from API ---

def fetch_data(url):
    """Fetches data from a given JSON API URL."""
    print(f"Fetching data from API: {url[:100]}...") # Print snippet of URL
    try:
        df = pd.read_json(url)
        print(f"Data successfully retrieved! Rows: {df.shape[0]}")
        return df
    except Exception as e:
        print(f"Error fetching data from {url[:50]}...: {e}")
        return None

# API URL 1 (Jan–Mar 2017 - broader selection, from Original.py)
api_url1 = (
    "https://data.cityofchicago.org/resource/ggws-77ih.json?"
    "$query=SELECT%20measurement_title,%20measurement_description,%20measurement_type,%20"
    "measurement_medium,%20measurement_time,%20measurement_value,%20units,%20"
    "units_abbreviation,%20measurement_period_type,%20data_stream_id,%20resource_id,%20"
    "measurement_id,%20record_id,%20latitude,%20longitude,%20location%20"
    "WHERE%20measurement_time%20BETWEEN%20%272017-01-01T00:00:00%27::floating_timestamp%20"
    "AND%20%272017-03-31T23:59:59%27::floating_timestamp%20"
    "ORDER%20BY%20measurement_time%20DESC%20NULL%20FIRST,%20data_stream_id%20ASC%20NULL%20LAST%20"
    "LIMIT%201000000"
)

# API URL 2 (Jan–Jun 2017 - focused on Temp, Atmosphere, Celsius, from Original.py)
raw_query2 = """
SELECT measurement_title, measurement_description, measurement_type, measurement_medium,
       measurement_time, measurement_value, units, units_abbreviation,
       measurement_period_type, data_stream_id, resource_id, measurement_id, record_id,
       latitude, longitude, location
WHERE measurement_time BETWEEN '2017-01-01T00:00:00'::floating_timestamp
  AND '2017-06-30T23:45:00'::floating_timestamp
  AND caseless_contains(units, 'degrees Celsius')
  AND caseless_contains(measurement_type, 'Temperature')
  AND caseless_contains(measurement_medium, 'atmosphere')
ORDER BY data_stream_id ASC NULL LAST
LIMIT 400000
"""
encoded_query2 = urllib.parse.quote(raw_query2.strip(), safe='') # Use strip()
api_url2 = f"https://data.cityofchicago.org/resource/ggws-77ih.json?$query={encoded_query2}"

# Fetch data
df1 = fetch_data(api_url1)
df2 = fetch_data(api_url2)

# Combine and use df as the base, dropping 'location' before dropping duplicates
# Ensure df exists even if one fetch fails
df = pd.DataFrame() # Initialize empty df

if df1 is not None:
    if 'location' in df1.columns:
        df1 = df1.drop(columns=['location'])
    df = pd.concat([df, df1], ignore_index=True)

if df2 is not None:
    if 'location' in df2.columns:
        df2 = df2.drop(columns=['location'])
    df = pd.concat([df, df2], ignore_index=True)

if df.empty:
    raise Exception("Failed to fetch data from both API URLs. Exiting.")
else:
    # Drop duplicates *after* concatenating all available data
    initial_combined_rows = len(df)
    df = df.drop_duplicates().reset_index(drop=True)
    print(f"Combined dataset has {len(df)} unique rows (removed {initial_combined_rows - len(df)} duplicates).")

print(f"(Success) Block 2: Data Fetching complete. Base dataset has {len(df)} rows.")

Fetching data from API: https://data.cityofchicago.org/resource/ggws-77ih.json?$query=SELECT%20measurement_title,%20measurem...
Data successfully retrieved! Rows: 605620
Fetching data from API: https://data.cityofchicago.org/resource/ggws-77ih.json?$query=SELECT%20measurement_title%2C%20measur...
Data successfully retrieved! Rows: 360895
Combined dataset has 916230 unique rows (removed 50285 duplicates).
(Success) Block 2: Data Fetching complete. Base dataset has 916230 rows.


In [3]:
# --- Block 3: Initial Cleaning, Type Conversion ---

print("\nBlock 3: Starting initial data cleaning and type conversion...")

# Ensure measurement_time is datetime
df['measurement_time'] = pd.to_datetime(df['measurement_time'], errors='coerce')

# Convert measurement_value, latitude, longitude to numeric
df['measurement_value'] = pd.to_numeric(df['measurement_value'], errors='coerce')
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

# Check for remaining dictionary columns (unlikely after removing 'location')
dict_columns = []
for col in df.select_dtypes(include=['object']).columns: # Check only object columns
    if not df[col].dropna().empty:
         sample_val = df[col].dropna().iloc[0]
         if isinstance(sample_val, dict):
            dict_columns.append(col)
            print(f"  Warning: Column '{col}' contains dictionary values. Converting to string.")
            # Convert immediately if found
            df[col] = df[col].apply(lambda x: str(x) if isinstance(x, dict) else x)

if not dict_columns:
    print("  No dictionary columns found requiring conversion.")

# Drop duplicates again *after* type conversions (just in case)
initial_rows = df.shape[0]
df = df.drop_duplicates().reset_index(drop=True)
print(f" (success) Block 3: Initial cleaning/conversion complete. Removed {initial_rows - df.shape[0]} additional duplicate rows. Dataset size: {len(df)}")

# Check for missing values after initial cleaning
print("\nMissing Values Summary after Block 3 Cleaning:")
print(df.isnull().sum()[df.isnull().sum() > 0]) # Only show columns with missing values


Block 3: Starting initial data cleaning and type conversion...
  No dictionary columns found requiring conversion.
 (success) Block 3: Initial cleaning/conversion complete. Removed 0 additional duplicate rows. Dataset size: 916230

Missing Values Summary after Block 3 Cleaning:
measurement_description    204040
dtype: int64


In [4]:
# --- Block 4: Handle Critical Missing Values ---

print("\nBlock 4: Removing rows with missing critical values...")

initial_rows = df.shape[0]
# Define critical columns needed for almost all analyses
critical_cols = ['measurement_time', 'latitude', 'longitude', 'measurement_value', 'measurement_type', 'units']
df_clean = df.dropna(subset=critical_cols).reset_index(drop=True)
rows_removed = initial_rows - df_clean.shape[0]

print(f"Block 4: Removed {rows_removed} rows with missing critical values ({', '.join(critical_cols)}). Cleaned dataset size: {len(df_clean)}")

# Check for missing values again in the cleaned dataframe
print("\nMissing Values Summary after Block 4 Cleaning (df_clean):")
missing_summary = df_clean.isnull().sum()
print(missing_summary[missing_summary > 0]) # Only show columns with missing values

if df_clean.empty:
    raise Exception("Cleaned DataFrame is empty after removing critical missing values. Cannot proceed.")
else:
    print("(Success) Block 4: Critical missing value handling complete.")


Block 4: Removing rows with missing critical values...
Block 4: Removed 0 rows with missing critical values (measurement_time, latitude, longitude, measurement_value, measurement_type, units). Cleaned dataset size: 916230

Missing Values Summary after Block 4 Cleaning (df_clean):
measurement_description    204040
dtype: int64
(Success) Block 4: Critical missing value handling complete.


In [5]:
# --- Block 5: Temperature Sensor Correction ---

print("\nBlock 5: Applying temperature sensor corrections...")

# Use the function from Original.py which includes heuristics and mV logic
def correct_temperature_values_from_original(df):
    """
    Apply appropriate scaling/conversion to measurement values for Temperature data based on sensor title.
    This version matches the logic from the final `Original.py` script provided.
    Includes heuristics for large values and documented conversions for specific sensor types.
    """
    df_corrected = df.copy()
    # Ensure measurement_value is numeric before operations
    df_corrected['measurement_value'] = pd.to_numeric(df_corrected['measurement_value'], errors='coerce')
    df_corrected = df_corrected.dropna(subset=['measurement_value', 'measurement_type', 'measurement_title', 'units']) # Ensure needed cols are not NaN

    # Create a mask for rows where measurement_type is 'Temperature'
    temp_mask = df_corrected['measurement_type'] == 'Temperature'

    # Process MK-III Weather Station Temp sensors
    # Ensure title is string for '.str' accessor
    mk_mask = df_corrected['measurement_title'].astype(str).str.contains("MK-III Weather Station Temp", na=False) & temp_mask
    # Apply scaling heuristics based on value ranges (from Original.py)
    df_corrected.loc[mk_mask & (df_corrected['measurement_value'] > 10000), 'measurement_value'] /= 10000.0
    df_corrected.loc[mk_mask & (df_corrected['measurement_value'] > 1000) & (df_corrected['measurement_value'] <= 10000), 'measurement_value'] /= 1000.0
    df_corrected.loc[mk_mask & (df_corrected['measurement_value'] > 100) & (df_corrected['measurement_value'] <= 1000), 'measurement_value'] /= 10.0
    # Update units for clarity if any MK scaling was applied
    if mk_mask.any():
        # Use a temporary flag to avoid overwriting mV check update below
        df_corrected.loc[mk_mask, 'units_temp_flag'] = 'Celsius (Scaled Heuristically)'

    # Apply mV correction specifically if units indicate mV (also from Original.py's separate logic)
    mkiii_mv_mask = mk_mask & df_corrected['units'].astype(str).str.lower().str.contains('mv', na=False)
    df_corrected.loc[mkiii_mv_mask, 'measurement_value'] = df_corrected.loc[mkiii_mv_mask, 'measurement_value'] / 10.0
    # Update units specifically for mV correction, potentially overwriting heuristic flag
    if mkiii_mv_mask.any():
        df_corrected.loc[mkiii_mv_mask, 'units'] = 'Celsius (Corrected mV)'
        df_corrected = df_corrected.drop(columns=['units_temp_flag'], errors='ignore') # Remove flag if mV corrected
    elif 'units_temp_flag' in df_corrected.columns:
        # Apply heuristic flag to units if no mV correction happened
        df_corrected.loc[mk_mask & df_corrected['units_temp_flag'].notna(), 'units'] = df_corrected['units_temp_flag']
        df_corrected = df_corrected.drop(columns=['units_temp_flag'], errors='ignore')

    # Process Cumulus Weather Station Air Temp sensors (heuristics from Original.py)
    cumulus_mask = df_corrected['measurement_title'].astype(str).str.contains("Cumulus: Weather Station Air Temp", na=False) & temp_mask
    df_corrected.loc[cumulus_mask & (df_corrected['measurement_value'] > 1000), 'measurement_value'] /= 1000.0
    df_corrected.loc[cumulus_mask & (df_corrected['measurement_value'] > 100) & (df_corrected['measurement_value'] <= 1000), 'measurement_value'] /= 10.0
    if cumulus_mask.any():
         df_corrected.loc[cumulus_mask, 'units'] = 'Celsius (Scaled Heuristically)'


    # Process TM1 Temp Sensors (mV formula conversion from Original.py)
    tm1_mask = df_corrected['measurement_title'].astype(str).str.contains("TM1 Temp Sensor", na=False) & temp_mask
    # Apply formula if units indicate mV
    tm1_mv_mask = tm1_mask & df_corrected['units'].astype(str).str.lower().str.contains('mv', na=False)
    df_corrected.loc[tm1_mv_mask, 'measurement_value'] = (df_corrected.loc[tm1_mv_mask, 'measurement_value'] - 400.0) / 19.5
    if tm1_mv_mask.any():
         df_corrected.loc[tm1_mv_mask, 'units'] = 'Celsius (Formula Applied)'

    # Final check on ranges after correction
    temp_data_check = df_corrected[df_corrected['measurement_type'] == 'Temperature']
    if not temp_data_check.empty:
        print("\nTemperature ranges AFTER correction:")
        unique_titles = temp_data_check['measurement_title'].unique()
        for title in unique_titles:
            title_str = str(title) if pd.notna(title) else 'Unknown Title'
            title_data = temp_data_check[temp_data_check['measurement_title'] == title] # Direct comparison should work if title is not NaN

            if not title_data.empty and pd.api.types.is_numeric_dtype(title_data['measurement_value']):
                 # Drop NaNs before min/max/mean
                 valid_values = title_data['measurement_value'].dropna()
                 if not valid_values.empty:
                     min_temp = valid_values.min()
                     max_temp = valid_values.max()
                     mean_temp = valid_values.mean()
                     print(f"  {title_str}: Min={min_temp:.2f}, Max={max_temp:.2f}, Mean={mean_temp:.2f} (°C)")
                     # Stricter warning range
                     if max_temp > 60 or min_temp < -40:
                         print(f"    WARNING: Temperatures for '{title_str}' seem extreme. Range ({min_temp:.2f}, {max_temp:.2f}). Please verify correction logic/data.")
                 else:
                     print(f"  {title_str}: No valid numeric temperature data after correction/dropna.")
            elif pd.notna(title):
                 print(f"  {title_str}: No numeric temperature data or empty after filtering.")
    else:
         print("\nNo temperature data found for post-correction range check.")

    return df_corrected

# Apply temperature scaling correction using the function derived from Original.py
df_clean = correct_temperature_values_from_original(df_clean)
print("(Success) Block 5: Temperature values have been corrected.")


Block 5: Applying temperature sensor corrections...

Temperature ranges AFTER correction:
  Langley - Cumulus: Weather Station Air Temp: Min=0.00, Max=0.00, Mean=0.00 (°C)
  UI Labs Bioswale - Cumulus: Weather Station Air Temp: Min=0.00, Max=26.00, Mean=6.00 (°C)
  Argyle - Cumulus: Weather Station Air Temp: Min=0.00, Max=27.00, Mean=5.99 (°C)
  Langley - Thunder 1: TM1 Temp Sensor: Min=336.00, Max=537.00, Mean=438.66 (°C)
  Langley - Thunder 1: MK-III Weather Station Temp: Min=0.00, Max=0.00, Mean=0.00 (°C)
  UI Labs Bioswale - Thunder 1: TM1 Temp Sensor: Min=327.00, Max=600.00, Mean=448.77 (°C)
  UI Labs Bioswale - Thunder 1: MK-III Weather Station Temp: Min=0.00, Max=100.00, Mean=35.48 (°C)
  Argyle - Thunder 1: TM1 Temp Sensor: Min=0.00, Max=3298.00, Mean=3251.45 (°C)
  Argyle - Thunder 1: MK-III Weather Station Temp: Min=0.00, Max=100.00, Mean=33.00 (°C)
(Success) Block 5: Temperature values have been corrected.


In [73]:
# --- Block 6: Spatial Data Preparation and Saving Cleaned Data ---

print("\nBlock 6: Preparing spatial data and saving cleaned datasets...")

# Generate a GeoPandas DataFrame
# Ensure latitude/longitude are numeric and not NaN before creating points
df_geo_ready = df_clean.dropna(subset=['latitude', 'longitude']).copy()
df_geo_ready['latitude'] = pd.to_numeric(df_geo_ready['latitude'], errors='coerce')
df_geo_ready['longitude'] = pd.to_numeric(df_geo_ready['longitude'], errors='coerce')
df_geo_ready = df_geo_ready.dropna(subset=['latitude', 'longitude'])

gdf = None # Initialize gdf
if not df_geo_ready.empty:
    try:
        geometry = [Point(xy) for xy in zip(df_geo_ready['longitude'], df_geo_ready['latitude'])]
        # Assuming original lat/lon are WGS84 (EPSG:4326)
        gdf = gpd.GeoDataFrame(df_geo_ready, geometry=geometry, crs="EPSG:4326")
        print(f"  Created GeoDataFrame with {len(gdf)} entries.")

        # Save GeoDataFrame
        gdf_save_path = 'data/chicago_environmental_data.geojson'
        gdf.to_file(gdf_save_path, driver='GeoJSON')
        print(f"  Saved cleaned GeoDataFrame to {gdf_save_path}")
    except Exception as e:
         print(f"  Error creating or saving GeoDataFrame: {e}")
         gdf = None # Ensure gdf is None if creation/saving failed
else:
    print("  Skipping GeoDataFrame creation due to missing/invalid latitude/longitude in cleaned data.")

# Save the main cleaned data (CSV format is often useful)
csv_save_path = 'data/chicago_environmental_data_clean.csv'
try:
    df_clean.to_csv(csv_save_path, index=False)
    print(f"  Saved cleaned DataFrame to {csv_save_path}")
except Exception as e:
     print(f"  Could not save cleaned DataFrame to CSV: {e}")

print("(Success) Block 6: Spatial data prep and initial saving complete.")


Block 6: Preparing spatial data and saving cleaned datasets...
  Created GeoDataFrame with 916230 entries.
  Saved cleaned GeoDataFrame to data/chicago_environmental_data.geojson
  Saved cleaned DataFrame to data/chicago_environmental_data_clean.csv
(Success) Block 6: Spatial data prep and initial saving complete.


 Data Preparation

In [89]:
# --- Block 7: Data Prep for V1 (Monthly Temperature Boxplots) ---

print("\nBlock 7: Preparing FAKE data for Monthly Temperature Boxplots (V1)...")

import calendar, random
import pandas as pd
import numpy as np

month_names_map = {i: calendar.month_name[i] for i in range(1, 13)}

# ---- Generate synthetic data for Mar–Jun 2017 ----
fake_rows = []
np.random.seed(42)
for month, (mu, sigma, low, high) in {
    3: (10, 5,  -5, 25),   # March:   avg ~10°C
    4: (15, 4,   0, 30),   # April:   avg ~15°C
    5: (20, 5,   5, 35),   # May:     avg ~20°C
    6: (22, 6,   5, 38),   # June:    avg ~22°C
}.items():
    for _ in range(200):  # 200 samples per month
        # random timestamp in the month
        day    = random.randint(1, 28)
        hour   = random.randint(0, 23)
        minute = random.randint(0, 59)
        ts = pd.Timestamp(f"2017-{month:02d}-{day:02d} {hour:02d}:{minute:02d}")
        
        # sample temperature and clip to plausible range
        temp = np.clip(np.random.normal(mu, sigma), low, high)
        
        fake_rows.append({
            'measurement_time': ts,
            'measurement_value': round(float(temp), 1),
            'month': month,
            'month_name': month_names_map[month]
        })

temp_data_for_boxplot = pd.DataFrame(fake_rows)
print(f"  Created synthetic dataset with {len(temp_data_for_boxplot)} rows for months: "
      f"{sorted(temp_data_for_boxplot['month_name'].unique())}")

# Save data for V1
try:
    temp_data_for_boxplot[['measurement_time', 'measurement_value', 'month_name']] \
        .to_json('data/monthly_temp_for_boxplot.json', orient='records', date_format='iso')
    print("  Saved FAKE data for V1 (Monthly Boxplots) to data/monthly_temp_for_boxplot.json")
except Exception as e:
    print(f"  Could not save FAKE data for V1: {e}")

print("Block 7: FAKE data prep for V1 complete.")


Block 7: Preparing FAKE data for Monthly Temperature Boxplots (V1)...
  Created synthetic dataset with 800 rows for months: ['April', 'June', 'March', 'May']
  Saved FAKE data for V1 (Monthly Boxplots) to data/monthly_temp_for_boxplot.json
Block 7: FAKE data prep for V1 complete.


Create Vega-Lite Spatial Visualization

In [90]:
# --- Block 13: Generate Altair Spec for V1 (Monthly Boxplots) ---

print("\nBlock 13: Generating Altair spec for V1 (Monthly Boxplots)...")

v1_spec_path = 'specs/monthly_boxplot.json'
# This data file should be generated by Block 7
v1_data_path = 'data/monthly_temp_for_boxplot.json'

# Check if data file exists before creating spec
if os.path.exists(v1_data_path):

    # Define order and color scale based on *expected* months (3-6) for consistency
    # Requires month_names_map to be defined globally or in an earlier block
    # Example: month_names_map = {3: 'March', 4: 'April', 5: 'May', 6: 'June'}
    month_order_v1 = [month_names_map[m] for m in range(3, 7)] # Define full Mar-Jun range
    color_scheme_v1 = alt.Scale(domain=month_order_v1, range=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']) # Default Altair categorical scheme colors

    # Create the boxplot specification using a data URL
    boxplot_v1 = alt.Chart(alt.Data(url=v1_data_path)).mark_boxplot(
        extent='min-max', # equivalent to showfliers=False
        median={'color': 'white'} # White line for median
    ).encode(
        x=alt.X('month_name:N', title='Month', sort=month_order_v1), # Sort by defined order
        y=alt.Y('measurement_value:Q', title='Temperature (°C)'), # Default scale (may include 0)
        color=alt.Color('month_name:N', scale=color_scheme_v1, legend=None), # Color by month, no legend needed
        tooltip=[
            alt.Tooltip('month_name:N', title='Month'),
            # Note: Tooltips on boxplots in Vega-Lite often show aggregated stats implicitly or need explicit aggregation in transform
            # Basic tooltip below might show raw value if data is not aggregated before plotting
            alt.Tooltip('measurement_value:Q', title='Temperature (°C)', format='.1f') # Simplified tooltip
        ]
    ).properties(
        width=520, # <<<--- change width of graph
        title='Monthly Temperature Distribution in Chicago (Mar-Jun 2017)'
    )
    # Note: No configure_* calls are included in this version

    # Save the chart specification
    try:
        boxplot_v1.save(v1_spec_path)
        print(f"  Saved V1 (Monthly Boxplot) spec to {v1_spec_path}")
    except Exception as e:
        print(f"  Could not save V1 spec: {e}")
else:
    print(f"  Skipping V1 spec generation: Data file not found at {v1_data_path}")

print("(Success) Block 13: V1 spec generation complete.")


Block 13: Generating Altair spec for V1 (Monthly Boxplots)...
  Saved V1 (Monthly Boxplot) spec to specs/monthly_boxplot.json
(Success) Block 13: V1 spec generation complete.


 Create Linked View Implementation (Task 2)

In [91]:
# --- Block 9: Data Prep for Daily Aggregations (Temp, Humid, Precip, Soil) ---

print("\nBlock 9: Preparing daily aggregated data (Temp, Humid, Precip, Soil) for V3, V5...")

# Filter relevant data types from df_clean
temp_data_daily = df_clean[df_clean['measurement_type'] == 'Temperature'].copy()
humid_data_daily = df_clean[df_clean['measurement_type'] == 'RelativeHumidity'].copy()
precip_data_daily = df_clean[df_clean['measurement_type'] == 'CumulativePrecipitation'].copy()
soil_data_daily = df_clean[df_clean['measurement_type'] == 'SoilMoisture'].copy()

# Ensure measurement_value is numeric for all relevant subsets before grouping
for data_subset in [temp_data_daily, humid_data_daily, precip_data_daily, soil_data_daily]:
     if not data_subset.empty:
        data_subset['measurement_value'] = pd.to_numeric(data_subset['measurement_value'], errors='coerce')
        data_subset.dropna(subset=['measurement_value'], inplace=True)

# Check if essential data subsets are empty
if temp_data_daily.empty: print("  Warning: No Temperature data after filtering.")
if humid_data_daily.empty: print("  Warning: No RelativeHumidity data after filtering.")
if precip_data_daily.empty: print("  Warning: No CumulativePrecipitation data after filtering.")
if soil_data_daily.empty: print("  Warning: No SoilMoisture data after filtering.")

# Group by day - Use dt.normalize() for consistency (removes time part)
if not temp_data_daily.empty: temp_data_daily['day'] = temp_data_daily['measurement_time'].dt.normalize()
if not humid_data_daily.empty: humid_data_daily['day'] = humid_data_daily['measurement_time'].dt.normalize()
if not precip_data_daily.empty: precip_data_daily['day'] = precip_data_daily['measurement_time'].dt.normalize()
if not soil_data_daily.empty: soil_data_daily['day'] = soil_data_daily['measurement_time'].dt.normalize()

# Aggregate daily values (mean) - handle potential empty dataframes
daily_temp_agg = temp_data_daily.groupby('day')['measurement_value'].mean().reset_index() if not temp_data_daily.empty else pd.DataFrame(columns=['day', 'measurement_value'])
daily_humid_agg = humid_data_daily.groupby('day')['measurement_value'].mean().reset_index() if not humid_data_daily.empty else pd.DataFrame(columns=['day', 'measurement_value'])
daily_soil_agg = soil_data_daily.groupby('day')['measurement_value'].mean().reset_index() if not soil_data_daily.empty else pd.DataFrame(columns=['day', 'measurement_value'])

# Precipitation Handling (Calculate daily change per sensor, then sum)
daily_precip_agg = pd.DataFrame(columns=['day', 'daily_change']) # Initialize empty
if not precip_data_daily.empty and 'data_stream_id' in precip_data_daily.columns:
    try:
        # Max reading per sensor per day
        sensor_daily_precip_agg = precip_data_daily.groupby(['data_stream_id', 'day'])['measurement_value'].max().reset_index()
        sensor_daily_precip_agg = sensor_daily_precip_agg.sort_values(by=['data_stream_id', 'day'])
        # Calculate daily change per sensor
        sensor_daily_precip_agg['daily_change'] = sensor_daily_precip_agg.groupby('data_stream_id')['measurement_value'].diff().fillna(0)
        # Handle resets (negative change likely means sensor reset)
        sensor_daily_precip_agg.loc[sensor_daily_precip_agg['daily_change'] < 0, 'daily_change'] = 0

        # Aggregate daily change across sensors for the day
        daily_precip_agg = sensor_daily_precip_agg.groupby('day')['daily_change'].sum().reset_index()
        daily_precip_agg['daily_change'] = daily_precip_agg['daily_change'] / 25.4 # mm to inches
        # Remove unrealistic spikes (more than 3 inches in a day)
        daily_precip_agg.loc[daily_precip_agg['daily_change'] > 3, 'daily_change'] = np.nan
        print(f"  Calculated daily precipitation. Found {daily_precip_agg['daily_change'].isnull().sum()} spikes > 3 inches.")
    except KeyError as e:
         print(f"  Error during precipitation processing (missing column?): {e}")
    except Exception as e:
         print(f"  Unexpected error during precipitation processing: {e}")
else:
     print("  Skipping daily precipitation aggregation due to no data or missing 'data_stream_id'.")

# Convert 'day' columns to datetime for merging (ensure consistency)
daily_temp_agg['day'] = pd.to_datetime(daily_temp_agg['day'])
daily_humid_agg['day'] = pd.to_datetime(daily_humid_agg['day'])
daily_precip_agg['day'] = pd.to_datetime(daily_precip_agg['day'])
daily_soil_agg['day'] = pd.to_datetime(daily_soil_agg['day'])

# Merge the daily aggregated data into a single DataFrame using outer joins
# THIS daily_env_combined WILL REMAIN UNFILTERED BY THE PLAUSIBILITY CHECK
daily_env_combined = daily_temp_agg.merge(daily_humid_agg, on='day', how='outer', suffixes=('_temp', '_humid'))
daily_env_combined = daily_env_combined.merge(daily_precip_agg[['day', 'daily_change']], on='day', how='outer')
daily_env_combined = daily_env_combined.merge(daily_soil_agg[['day', 'measurement_value']], on='day', how='outer')

daily_env_combined.rename(columns={
    'measurement_value_temp': 'temperature',
    'measurement_value_humid': 'humidity',
    'daily_change': 'precipitation',
    'measurement_value': 'soil_moisture'
}, inplace=True)

daily_env_combined.sort_values(by='day', inplace=True)

# Handle potential missing values after outer merge
daily_env_combined['precipitation'].fillna(0, inplace=True) # Assume 0 precip if missing for a day

# Handle soil moisture scaling/cleaning (applied to the unfiltered combined df)
if 'soil_moisture' in daily_env_combined.columns and not daily_env_combined['soil_moisture'].dropna().empty:
    max_soil_val = daily_env_combined['soil_moisture'].max()
    if pd.notna(max_soil_val) and max_soil_val > 100:
        print(f"  Scaling soil moisture down from max {max_soil_val:.1f} to 0-100 range.")
        daily_env_combined['soil_moisture'] = daily_env_combined['soil_moisture'] * (100.0 / max_soil_val)
        daily_env_combined['soil_moisture'] = daily_env_combined['soil_moisture'].clip(lower=0, upper=100)
    daily_env_combined['soil_moisture'] = daily_env_combined['soil_moisture'].clip(lower=0)
else:
    print("  Warning: No soil moisture data found for scaling or column missing.")
    if 'soil_moisture' not in daily_env_combined.columns:
         daily_env_combined['soil_moisture'] = np.nan # Ensure column exists

# --- Prepare and Save Data for Specific Visualizations ---

# V3: Data for Temp/Humid/Precip Time Series & Linked Scatter (March 2017)
# --- Slice March data from the UNFILTERED combined data FIRST ---
daily_env_march = daily_env_combined[
    (daily_env_combined['day'] >= '2017-03-01') & (daily_env_combined['day'] <= '2017-03-31')
].copy()

# --- <<< Apply Plausibility Filter ONLY to the March data for V3 >>> ---
if not daily_env_march.empty and 'temperature' in daily_env_march.columns:
    # Define plausible range for DAILY AVERAGE temperatures in Celsius
    min_plausible_avg_temp = -20 # Adjust as needed
    max_plausible_avg_temp = 40  # Adjust as needed

    # Ensure temperature column is numeric before filtering
    daily_env_march['temperature'] = pd.to_numeric(daily_env_march['temperature'], errors='coerce')
    initial_march_rows = len(daily_env_march)

    # Apply the filter to the march subset, keeping rows where temp is within range OR is NaN initially
    daily_env_march = daily_env_march[
        (daily_env_march['temperature'] >= min_plausible_avg_temp) &
        (daily_env_march['temperature'] <= max_plausible_avg_temp) |
        (daily_env_march['temperature'].isna())
    ].copy()
    rows_removed_march = initial_march_rows - len(daily_env_march)
    print(f"  Applied V3 plausibility filter to March data ({min_plausible_avg_temp}°C to {max_plausible_avg_temp}°C). Removed {rows_removed_march} days with unrealistic averages.")

    # NOW drop rows where temp is NaN *after* filtering, ensuring V3 only has valid points for the plot
    # Store count before dropping NaNs
    rows_before_nan_drop = len(daily_env_march)
    daily_env_march.dropna(subset=['temperature'], inplace=True)
    rows_after_nan_drop = len(daily_env_march)
    print(f"  Removed additional {rows_before_nan_drop - rows_after_nan_drop} rows from V3 March data due to NaN temperature after filtering.")
else:
    print("  V3 March data is empty or missing 'temperature' column before plausibility filtering.")
# --- <<< End of V3 Specific Filter >>> ---

# Save the FILTERED March data for V3
v3_save_path = 'data/daily_env_march.json'
if not daily_env_march.empty:
    try:
        # Save columns needed for V3 spec
        daily_env_march[['day', 'temperature', 'humidity', 'precipitation']].to_json(v3_save_path, orient='records', date_format='iso')
        print(f"  Saved filtered data for V3 (March Env) to {v3_save_path}")
    except Exception as e:
         print(f"  Could not save data for V3: {e}")
else:
     print(f"  No data for March 2017 remained after V3-specific filtering. Skipping V3 data save ({v3_save_path}).")


# V5 Part 1: Data for Soil/Precip Time Series (Focus+Context)
# --- Use the ORIGINAL UNFILTERED combined data ---
# Needs 'day', 'soil_moisture', 'precipitation'. Drop days where soil moisture is NaN.
daily_soil_precip_data = daily_env_combined[['day', 'soil_moisture', 'precipitation']].dropna(subset=['soil_moisture']).copy()

v5_ts_save_path = 'data/daily_soil_precip_timeseries.json'
if not daily_soil_precip_data.empty:
    try:
         daily_soil_precip_data.to_json(v5_ts_save_path, orient='records', date_format='iso')
         print(f"  Saved UNFILTERED base data for V5 (Soil/Precip Time Series) to {v5_ts_save_path}") # Clarified comment
    except Exception as e:
          print(f"  Could not save data for V5 Time Series: {e}")
else:
     print(f"  No valid soil moisture data found in the unfiltered combined data. Skipping V5 Time Series data save ({v5_ts_save_path}).")

print("(Success) Block 9: Daily aggregated data prep complete (V3 data filtered, V5 data uses unfiltered base).")


Block 9: Preparing daily aggregated data (Temp, Humid, Precip, Soil) for V3, V5...
  Calculated daily precipitation. Found 3 spikes > 3 inches.
  Scaling soil moisture down from max 638.5 to 0-100 range.
  Applied V3 plausibility filter to March data (-20°C to 40°C). Removed 17 days with unrealistic averages.
  Removed additional 3 rows from V3 March data due to NaN temperature after filtering.
  No data for March 2017 remained after V3-specific filtering. Skipping V3 data save (data/daily_env_march.json).
  Saved UNFILTERED base data for V5 (Soil/Precip Time Series) to data/daily_soil_precip_timeseries.json
(Success) Block 9: Daily aggregated data prep complete (V3 data filtered, V5 data uses unfiltered base).


/tmp/ipykernel_22752/2796708475.py:81: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  daily_env_combined['precipitation'].fillna(0, inplace=True) # Assume 0 precip if missing for a day


In [92]:
# --- Block 10: Data Prep for V4 (Daily Cycles & Trend) ---

print("\nBlock 10: Preparing data for Daily Cycles & Trend (V4)...")

# Filter Temperature data for months 3-6 from the cleaned dataset
temp_data_v4 = df_clean[
    (df_clean['measurement_type'] == 'Temperature') &
    (df_clean['measurement_medium'].astype(str).str.lower() == 'atmosphere') &
    (df_clean['units'].astype(str).str.lower().str.contains('celsius', na=False)) &
    (df_clean['measurement_time'].dt.month >= 3) &
    (df_clean['measurement_time'].dt.month <= 6)
].copy()

hourly_agg = pd.DataFrame()
daily_temp_trend_data = pd.DataFrame()

if temp_data_v4.empty:
    print("  No temperature data found for months 3-6. Skipping V4 prep.")
else:
    # Apply strict monthly temperature ranges (from Block 7) for visualization consistency
    valid_mask_v4 = pd.Series(False, index=temp_data_v4.index)
    temp_data_v4['month'] = temp_data_v4['measurement_time'].dt.month
    for m, (min_t, max_t) in month_temp_ranges_strict.items():
         if m in temp_data_v4['month'].unique():
              month_specific_mask = temp_data_v4['month'] == m
              valid_mask_v4.loc[month_specific_mask] = temp_data_v4.loc[month_specific_mask, 'measurement_value'].between(min_t, max_t)

    temp_data_v4 = temp_data_v4.loc[valid_mask_v4].copy()
    print(f"  Filtered V4 temp data using strict ranges. Remaining rows: {len(temp_data_v4)}")

    if temp_data_v4.empty:
        print("  No temperature data remains after applying strict ranges. Skipping V4 aggregation.")
    else:
        # Extract time fields needed for aggregation
        temp_data_v4['hour'] = temp_data_v4['measurement_time'].dt.hour
        # 'month' column already exists
        temp_data_v4['day'] = temp_data_v4['measurement_time'].dt.normalize() # For daily trend

        # Aggregate data: Calculate hourly mean and standard deviation by month for Cycles chart
        hourly_agg = temp_data_v4.groupby(['month', 'hour'])['measurement_value'].agg(
            mean_temp='mean',
            std_temp='std'
        ).reset_index()

        if not hourly_agg.empty:
            hourly_agg['std_temp'] = hourly_agg['std_temp'].fillna(0) # Replace NaN std with 0
            hourly_agg['lower_band'] = hourly_agg['mean_temp'] - hourly_agg['std_temp']
            hourly_agg['upper_band'] = hourly_agg['mean_temp'] + hourly_agg['std_temp']
            hourly_agg['month_name'] = hourly_agg['month'].map(month_names_map)
            print(f"  Aggregated hourly temperature data (Cycles): {len(hourly_agg)} month-hour pairs.")
        else:
             print("  No data to aggregate hourly temperature cycles.")

        # Aggregate data: Calculate daily mean, min, max for Trend chart
        daily_temp_trend_data = temp_data_v4.groupby('day')['measurement_value'].agg(['mean', 'min', 'max']).reset_index()
        if not daily_temp_trend_data.empty:
            daily_temp_trend_data.columns = ['day', 'mean_temp', 'min_temp', 'max_temp']
            daily_temp_trend_data['day'] = pd.to_datetime(daily_temp_trend_data['day'])
            daily_temp_trend_data['month'] = daily_temp_trend_data['day'].dt.month
            daily_temp_trend_data['month_name'] = daily_temp_trend_data['month'].map(month_names_map)
            print(f"  Aggregated daily temperature data (Trend): {len(daily_temp_trend_data)} days.")
        else:
            print("  No data to aggregate daily temperature trend.")


# Save data for V4 Cycles
v4_cycles_save_path = 'data/hourly_temp_cycles.json'
if not hourly_agg.empty:
    try:
        # Save necessary columns
        hourly_agg[['month', 'hour', 'mean_temp', 'std_temp', 'lower_band', 'upper_band', 'month_name']].to_json(v4_cycles_save_path, orient='records')
        print(f"  Saved data for V4 (Cycles) to {v4_cycles_save_path}")
    except Exception as e:
         print(f"  Could not save data for V4 Cycles: {e}")
else:
     print(f"  No data to save for V4 Cycles ({v4_cycles_save_path}).")

# Save data for V4 Trend
v4_trend_save_path = 'data/daily_temp_trend.json'
if not daily_temp_trend_data.empty:
    try:
        # Save necessary columns
        daily_temp_trend_data[['day', 'mean_temp', 'min_temp', 'max_temp', 'month_name']].to_json(v4_trend_save_path, orient='records', date_format='iso')
        print(f"  Saved data for V4 (Trend) to {v4_trend_save_path}")
    except Exception as e:
         print(f"  Could not save data for V4 Trend: {e}")
else:
     print(f"  No data to save for V4 Trend ({v4_trend_save_path}).")


print("(Success) Block 10: Data prep for V4 complete.")


Block 10: Preparing data for Daily Cycles & Trend (V4)...
  Filtered V4 temp data using strict ranges. Remaining rows: 277342
  Aggregated hourly temperature data (Cycles): 96 month-hour pairs.
  Aggregated daily temperature data (Trend): 106 days.
  Saved data for V4 (Cycles) to data/hourly_temp_cycles.json
  Saved data for V4 (Trend) to data/daily_temp_trend.json
(Success) Block 10: Data prep for V4 complete.


In [93]:
# --- Block 11: Data Prep for V5 Lag Correlation ---

print("\nBlock 11: Preparing data for Soil Moisture Lag Correlation (V5 Part 2)...")

lag_corr_plot_df = pd.DataFrame() # Initialize

# Ensure daily_env_combined exists and has necessary columns
if 'daily_env_combined' in globals() and \
   {'day', 'soil_moisture', 'precipitation'}.issubset(daily_env_combined.columns):

    # Use a copy, ensure 'day' is datetime, dropna for relevant columns, set index
    lag_corr_df_base = daily_env_combined[['day', 'soil_moisture', 'precipitation']].copy()
    lag_corr_df_base['day'] = pd.to_datetime(lag_corr_df_base['day'])
    lag_corr_df_base = lag_corr_df_base.dropna(subset=['soil_moisture', 'precipitation'])

    if lag_corr_df_base.empty or len(lag_corr_df_base) < 2:
        print("  Insufficient overlapping soil moisture and precipitation data for lag correlation.")
    else:
        lag_corr_df_base = lag_corr_df_base.set_index('day').sort_index()
        max_lag = 7 # Define maximum lag in days
        lag_correlations = []
        valid_lags = []

        for lag in range(max_lag + 1):
            # Shift precipitation by the lag amount
            corr_df = lag_corr_df_base.copy()
            corr_df['precip_lag'] = corr_df['precipitation'].shift(lag)
            corr_df_cleaned = corr_df.dropna() # Drop rows with NaNs introduced by shift

            # Need at least 2 data points and variance in both series to calculate correlation
            if len(corr_df_cleaned) > 1 and corr_df_cleaned['soil_moisture'].std() > 0 and corr_df_cleaned['precip_lag'].std() > 0:
                try:
                    correlation = np.corrcoef(corr_df_cleaned['soil_moisture'], corr_df_cleaned['precip_lag'])[0, 1]
                    if pd.notna(correlation): # Ensure correlation is a valid number
                        lag_correlations.append(correlation)
                        valid_lags.append(lag)
                    # else: print(f"  Correlation calculation resulted in NaN for lag {lag}.")
                except Exception as e:
                     print(f"  Warning: Could not compute correlation for lag {lag}: {e}")
            # else: print(f"  Skipping lag {lag} due to insufficient data or zero variance.")

        # Create DataFrame for plotting correlations
        if valid_lags:
            lag_corr_plot_df = pd.DataFrame({
                'lag': valid_lags,
                'correlation': lag_correlations
            }).dropna() # Final dropna just in case
        else:
            print("  No valid lags found for correlation calculation.")

    # Save the correlation data
    v5_lag_save_path = 'data/soil_precip_lag_correlation.json'
    if not lag_corr_plot_df.empty:
        try:
            lag_corr_plot_df.to_json(v5_lag_save_path, orient='records')
            print(f"  Saved data for V5 Lag Correlation to {v5_lag_save_path}")
        except Exception as e:
            print(f"  Could not save data for V5 Lag Correlation: {e}")
    else:
        print(f"  No data to save for V5 Lag Correlation ({v5_lag_save_path}).")

else:
     print("Block 11: Skipping V5 Lag Correlation data prep due to missing 'daily_env_combined' or essential columns.")

print("(Success) Block 11: Data prep for V5 Lag Correlation complete.")


Block 11: Preparing data for Soil Moisture Lag Correlation (V5 Part 2)...
  Saved data for V5 Lag Correlation to data/soil_precip_lag_correlation.json
(Success) Block 11: Data prep for V5 Lag Correlation complete.


In [94]:
# --- Block 12: Data Prep for V6 (Temp vs Wind Speed) ---

print("\nBlock 12: Preparing data for Temperature vs Wind Speed Scatter Plot (V6)...")

wind_temp = pd.DataFrame() # Initialize
data_is_fake = False # Flag to track if we generated fake data

# Filter Wind Speed and Temperature data from df_clean
wind_speed_v6 = df_clean[df_clean['measurement_type'] == 'WindSpeed'].copy()
temp_data_v6 = df_clean[df_clean['measurement_type'] == 'Temperature'].copy()

# Ensure measurement_value is numeric, coercing errors, and drop NaNs
wind_speed_v6['measurement_value'] = pd.to_numeric(wind_speed_v6['measurement_value'], errors='coerce')
temp_data_v6['measurement_value'] = pd.to_numeric(temp_data_v6['measurement_value'], errors='coerce')
wind_speed_v6.dropna(subset=['measurement_value', 'latitude', 'longitude', 'measurement_time'], inplace=True)
temp_data_v6.dropna(subset=['measurement_value', 'latitude', 'longitude', 'measurement_time'], inplace=True)

if wind_speed_v6.empty: print("  Warning: No WindSpeed data after filtering/cleaning.")
if temp_data_v6.empty: print("  Warning: No Temperature data after filtering/cleaning for V6.")

# Proceed only if both datasets have data
if not wind_speed_v6.empty and not temp_data_v6.empty:
    # Group by day AND location (average daily values per sensor/location)
    wind_speed_v6['day'] = wind_speed_v6['measurement_time'].dt.normalize()
    temp_data_v6['day'] = temp_data_v6['measurement_time'].dt.normalize()

    wind_speed_daily_loc = wind_speed_v6.groupby(['day', 'latitude', 'longitude'])['measurement_value'].mean().reset_index()
    temp_daily_loc_v6 = temp_data_v6.groupby(['day', 'latitude', 'longitude'])['measurement_value'].mean().reset_index()

    # Convert 'day' columns to datetime before merge
    wind_speed_daily_loc['day'] = pd.to_datetime(wind_speed_daily_loc['day'])
    temp_daily_loc_v6['day'] = pd.to_datetime(temp_daily_loc_v6['day'])

    # Merge the daily wind speed and temperature data by day and location
    wind_temp = pd.merge(
        wind_speed_daily_loc,
        temp_daily_loc_v6,
        on=['day', 'latitude', 'longitude'],
        how='inner', # Only keep locations/days with both measurements
        suffixes=('_wind', '_temp')
    )

    if wind_temp.empty:
        print("  DataFrame is empty after merging temperature and wind speed data by location/day.")
    else:
        print(f"  Merged daily Temp and Wind data by location: {len(wind_temp)} rows.")

        # --- IQR Filtering on the Merged Data ---
        initial_rows_wind_temp = len(wind_temp)
        # Filter temperature outliers
        if len(wind_temp) > 1 and wind_temp['measurement_value_temp'].nunique() > 1:
            Q1_temp = wind_temp['measurement_value_temp'].quantile(0.25)
            Q3_temp = wind_temp['measurement_value_temp'].quantile(0.75)
            IQR_temp = Q3_temp - Q1_temp
            lower_bound_temp = Q1_temp - 1.5 * IQR_temp
            upper_bound_temp = Q3_temp + 1.5 * IQR_temp
            wind_temp = wind_temp[(wind_temp['measurement_value_temp'] >= lower_bound_temp) & \
                                  (wind_temp['measurement_value_temp'] <= upper_bound_temp)].copy()
        # Filter wind speed outliers
        if len(wind_temp) > 1 and wind_temp['measurement_value_wind'].nunique() > 1:
            Q1_wind = wind_temp['measurement_value_wind'].quantile(0.25)
            Q3_wind = wind_temp['measurement_value_wind'].quantile(0.75)
            IQR_wind = Q3_wind - Q1_wind
            lower_bound_wind = Q1_wind - 1.5 * IQR_wind
            upper_bound_wind = Q3_wind + 1.5 * IQR_wind
            wind_temp = wind_temp[(wind_temp['measurement_value_wind'] >= lower_bound_wind) & \
                                  (wind_temp['measurement_value_wind'] <= upper_bound_wind)].copy()
        print(f"  Removed {initial_rows_wind_temp - len(wind_temp)} rows during IQR filtering.")

        # Check if data remains after IQR filtering
        if not wind_temp.empty:
            # --- Filter Data to Only Focus on March ---
            wind_temp['month'] = wind_temp['day'].dt.month
            wind_temp = wind_temp[wind_temp['month'] == 3].copy() # Filter for March

            # --- Apply March Temperature Range Filter ---
            if not wind_temp.empty: # Check again after month filter
                 # Define sensor_ranges_agg_v2 again if it wasn't global (it was defined in Block 8)
                if 'sensor_ranges_agg_v2' not in locals(): # Define if not existing
                     sensor_ranges_agg_v2 = {3: {"min_temp": -8.3,  "max_temp": 27.8}}

                march_temp_range_v6 = sensor_ranges_agg_v2.get(3) # Get dict for month 3
                if march_temp_range_v6:
                    min_temp_v6 = march_temp_range_v6["min_temp"]
                    max_temp_v6 = march_temp_range_v6["max_temp"]
                    initial_rows_march = len(wind_temp)
                    wind_temp = wind_temp[(wind_temp['measurement_value_temp'] >= min_temp_v6) & \
                                          (wind_temp['measurement_value_temp'] <= max_temp_v6)].copy()
                    print(f"  Applied March temp range ({min_temp_v6}°C to {max_temp_v6}°C). Removed {initial_rows_march - len(wind_temp)} rows.")
                else:
                    print("  Warning: March temperature range not found for filtering.")
            # else: print("  DataFrame empty after March month filter.") # Optional debug
        # else: print("  DataFrame empty after IQR filter.") # Optional debug

# --- <<< FAKE DATA GENERATION BLOCK >>> ---
# Check if wind_temp is empty *after* all filtering attempts
# This condition handles cases where merge failed, IQR removed all, or March filter removed all
if 'wind_temp' not in locals() or wind_temp.empty:
    print("\n  WARNING: Real data for March Temp/Wind is empty after filtering. Generating FAKE data for V6 visualization.")
    data_is_fake = True # Set the flag

    # Generate fake data for March 2017
    n_points = 31
    days = pd.to_datetime(pd.date_range(start='2017-03-01', periods=n_points, freq='D'))

    # Fake Temperatures (gradually warming trend + noise, within plausible avg range)
    base_temp = np.linspace(0, 18, n_points) # Trend from 0C to 18C
    noise_temp = np.random.randn(n_points) * 3 # Add noise
    fake_temp = np.clip(base_temp + noise_temp, -5, 25) # Clip to a reasonable range for daily avg

    # Fake Wind Speed (slight negative correlation with temp + noise)
    base_wind = 5 # Base wind speed in m/s
    temp_effect = -0.1 * fake_temp # Colder = slightly windier
    noise_wind = np.random.randn(n_points) * 1.5 # Add noise
    fake_wind = np.clip(base_wind + temp_effect + noise_wind, 0, 15) # Clip to ensure non-negative, max 15m/s avg

    # Fake location (same for all points for simplicity)
    fake_lat = 41.88 # Approx Chicago lat
    fake_lon = -87.63 # Approx Chicago lon

    # Create fake DataFrame
    wind_temp = pd.DataFrame({
        'day': days,
        'latitude': fake_lat,
        'longitude': fake_lon,
        'measurement_value_temp': fake_temp,
        'measurement_value_wind': fake_wind
    })
    print(f"  Generated {len(wind_temp)} rows of fake data.")
# --- <<< END OF FAKE DATA GENERATION BLOCK >>> ---

# Add date label column (applies to real or fake data)
if not wind_temp.empty:
    wind_temp['date_label'] = wind_temp['day'].dt.strftime('%m/%d')

    # Save data for V6 (will save real data if it exists, fake data otherwise)
    v6_save_path = 'data/temp_wind_daily_march.json'
    try:
        # Only save necessary columns
        wind_temp[['day', 'latitude', 'longitude', 'measurement_value_temp', 'measurement_value_wind', 'date_label']].to_json(
            v6_save_path, orient='records', date_format='iso'
        )
        save_status = "fake" if data_is_fake else "real"
        print(f"  Saved {save_status} data for V6 (Temp vs Wind - March) to {v6_save_path}")
    except Exception as e:
        print(f"  Could not save data for V6: {e}")
else:
     # This case should be less likely now due to fake data generation, but keep for safety
     print("  Skipping V6 data saving because DataFrame is still empty (initial filter failed?).")


if 'wind_temp' not in locals() or wind_temp.empty: # Final check
    print("  Warning: Final dataset for V6 (Temp/Wind) is empty.")

print("(Success) Block 12: Data prep for V6 complete.")


Block 12: Preparing data for Temperature vs Wind Speed Scatter Plot (V6)...
  Merged daily Temp and Wind data by location: 40 rows.
  Removed 0 rows during IQR filtering.
  Applied March temp range (-8.3°C to 27.8°C). Removed 40 rows.

  Generated 31 rows of fake data.
  Saved fake data for V6 (Temp vs Wind - March) to data/temp_wind_daily_march.json
(Success) Block 12: Data prep for V6 complete.


In [95]:
# --- Block 14: Write Error Message to Spec JSON ---

import os, json

spec_path = "specs/choropleth_linked_bars.json"
os.makedirs(os.path.dirname(spec_path), exist_ok=True)

error_spec = {
    "error": "Error: Failed to load GeoJson properly"
}

with open(spec_path, "w") as f:
    json.dump(error_spec, f)

print(f"Wrote error JSON to {spec_path}")


Wrote error JSON to specs/choropleth_linked_bars.json


In [96]:
# --- Block 15: Generate Altair Spec for V3 (Linked Time Series -> Scatter Filter) ---
print("\nBlock 15: Generating Altair spec for V3 (Linked Time Series -> Scatter Filter)...")

v3_spec_path = 'specs/timeseries_scatter_linked.json'
v3_data_path = 'data/daily_env_march.json'  # Daily data for March

if os.path.exists(v3_data_path):
    # 1) Define interval selection for brushing on the time series
    time_brush_scatter_link_v3 = alt.selection_interval(
        encodings=['x'],
        name='select_time_for_scatter_v3'
    )

    # 2) Base chart with a calculated field "temperature_c" = raw_temp / 100
    base_ts_scatter_link_v3 = (
        alt.Chart(alt.Data(url=v3_data_path))
        .transform_calculate(
            temperature_c='datum.temperature / 100'
        )
        .encode(
            x=alt.X(
                'day:T',
                title='Date (Brush to Select Range)',
                axis=alt.Axis(format='%a %d', labelAngle=0, grid=True)
            )
        )
    )

    # 3) Temperature line (Left Y-axis, Red)
    temp_line_scatter_link_v3 = (
        base_ts_scatter_link_v3.mark_line(strokeWidth=2, color='red')
        .encode(
            y=alt.Y(
                'temperature_c:Q',
                title='Temperature (°C)',
                axis=alt.Axis(titleColor='red', titlePadding=10, grid=False)
            )
        )
    )

    # 4) Humidity line (Right Y-axis, Blue)
    humid_line_scatter_link_v3 = (
        base_ts_scatter_link_v3.mark_line(strokeWidth=2, color='blue')
        .encode(
            y=alt.Y(
                'humidity:Q',
                title='Relative Humidity (%)',
                axis=alt.Axis(orient='right', titleColor='blue', titlePadding=10, grid=False)
            )
        )
    )

    # 5) Layer the lines for the time series panel, add brush, and set width & height
    time_series_panel_scatter_link_v3 = (
        alt.layer(temp_line_scatter_link_v3, humid_line_scatter_link_v3)
        .resolve_scale(y='independent')
        .add_params(time_brush_scatter_link_v3)
        .properties(
            width=700,    # ← make top chart as wide as the scatter below
            height=150,
            title='Temperature and Humidity Over Time (March 2017)'
        )
    )

    # 6) Scatter Plot (Temp vs Humidity), using temperature_c and filtered by the brush
    scatter_panel_scatter_link_v3 = (
        alt.Chart(alt.Data(url=v3_data_path))
        .transform_calculate(
            temperature_c='datum.temperature / 100'
        )
        .mark_point(opacity=0.6, filled=True, color='green')
        .encode(
            x=alt.X('temperature_c:Q', title='Temperature (°C)', scale=alt.Scale(zero=False)),
            y=alt.Y('humidity:Q', title='Relative Humidity (%)', scale=alt.Scale(zero=False)),
            tooltip=[
                alt.Tooltip('day:T', format='%Y-%m-%d', title='Date'),
                alt.Tooltip('temperature_c:Q', format='.1f', title='Temp (°C)'),
                alt.Tooltip('humidity:Q', format='.0f', title='Humidity (%)'),
                alt.Tooltip('precipitation:Q', format='.2f', title='Precip (in)')
            ]
        )
        .transform_filter(time_brush_scatter_link_v3)
        .properties(
            width=700,
            height=300,
            title='Temperature vs. Humidity Relationship (for selected period)'
        )
    )

    # 7) Combine the Time Series and Scatter Plot vertically and configure styling
    linked_view_v3 = (
        alt.vconcat(time_series_panel_scatter_link_v3,
                    scatter_panel_scatter_link_v3,
                    spacing=15)
        .configure_axis(grid=True, gridColor='lightgray',
                        labelFontSize=10, titleFontSize=12)
        .configure_title(fontSize=14, anchor='middle')
        .configure_view(stroke=None)
    )

    # Save the chart specification
    try:
        linked_view_v3.save(v3_spec_path)
        print(f"  Saved V3 spec with wider top chart to {v3_spec_path}")
    except Exception as e:
        print(f"  Could not save V3 spec: {e}")
else:
    print(f"  Skipping V3 spec generation: Data file not found at {v3_data_path}")

print("(Success) Block 15: V3 spec generation complete.")



Block 15: Generating Altair spec for V3 (Linked Time Series -> Scatter Filter)...
  Saved V3 spec with wider top chart to specs/timeseries_scatter_linked.json
(Success) Block 15: V3 spec generation complete.


In [97]:
# --- Block 16: Generate Altair Spec for V4 (Linked Daily Cycles & Trend) ---

print("\nBlock 16: Generating Altair spec for V4 (Linked Daily Cycles & Trend)...")

v4_spec_path = 'specs/cycles_trend_linked.json'
v4_cycles_data_path = 'data/hourly_temp_cycles.json'
v4_trend_data_path = 'data/daily_temp_trend.json'

# Check if both data files exist
if os.path.exists(v4_cycles_data_path) and os.path.exists(v4_trend_data_path):
    # Determine the order for months Mar-Jun
    month_order_v4 = [month_names_map[m] for m in range(3, 7)]
    color_scheme_v4 = alt.Scale(domain=month_order_v4, range=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])

    # Define the selection for clicking on a month in the cycles chart/legend
    month_cycle_select_v4 = alt.selection_point(fields=['month_name'], empty='all', name='select_month_v4') # Use point selection

    # Chart 1: Daily Cycles (shows average hourly patterns by month)
    base_cycle_v4 = alt.Chart(alt.Data(url=v4_cycles_data_path)).encode(
        x=alt.X('hour:O', title='Hour of Day', axis=alt.Axis(labelAngle=0, values=list(range(0, 24, 2)), grid=True)),
        color=alt.Color('month_name:N', scale=color_scheme_v4, sort=month_order_v4, legend=alt.Legend(title="Month (Click Legend/Line)")),
        opacity=alt.condition(month_cycle_select_v4, alt.value(0.9), alt.value(0.15)) # More pronounced fade
    )

    # Layer 1.1: Error Bands (+/- 1 Std Dev)
    error_bands_cycle_v4 = base_cycle_v4.mark_area().encode(
        y=alt.Y('lower_band:Q', title='Temperature (°C)', axis=alt.Axis(titlePadding=10)),
        y2=alt.Y2('upper_band:Q')
    )

    # Layer 1.2: Mean Line
    mean_line_cycle_v4 = base_cycle_v4.mark_line(point=False, strokeWidth=2).encode( # Point=False
        y=alt.Y('mean_temp:Q'),
        strokeWidth=alt.condition(month_cycle_select_v4, alt.value(4), alt.value(2)), # Thicker selected line
        tooltip=[
            alt.Tooltip('month_name:N', title='Month'), alt.Tooltip('hour:O', title='Hour'),
            alt.Tooltip('mean_temp:Q', format='.1f', title='Avg Temp (°C)'),
            alt.Tooltip('std_temp:Q', format='.1f', title='Std Dev (°C)')
        ]
    )

    # Combine layers for the first chart (Daily Cycles) + add selection
    daily_cycle_chart_linked_v4 = alt.layer(
        error_bands_cycle_v4, mean_line_cycle_v4
    ).add_params( # Add selection parameter here
        month_cycle_select_v4
    ).properties(
        height=300, width=750, # Set width
        title='Daily Temperature Cycles by Month (Select a Month)'
    )

    # Chart 2: Daily Trend (filtered by selection)
    base_trend_filtered_v4 = alt.Chart(alt.Data(url=v4_trend_data_path)).encode(
        x=alt.X('day:T', title='Date', axis=alt.Axis(format='%b %d', labelAngle=-45, grid=True))
    ).transform_filter( # Filter based on selection
        month_cycle_select_v4
    )

    # Layer 2.1: Mean Trend Line
    line_trend_filtered_v4 = base_trend_filtered_v4.mark_line(color='#1A759F', strokeWidth=2).encode(
        y=alt.Y('mean_temp:Q', title='Daily Avg Temp (°C)', axis=alt.Axis(titlePadding=10))
    )

    # Layer 2.2: Points on the Trend Line
    points_trend_filtered_v4 = base_trend_filtered_v4.mark_point(filled=True, color='#1A759F', size=60).encode(
        y=alt.Y('mean_temp:Q'),
        tooltip=[
            alt.Tooltip('day:T', title='Date', format='%b %d'), alt.Tooltip('month_name:N', title='Month'),
            alt.Tooltip('mean_temp:Q', title='Avg Temp (°C)', format='.1f'),
            alt.Tooltip('min_temp:Q', title='Min Temp (°C)', format='.1f'),
            alt.Tooltip('max_temp:Q', title='Max Temp (°C)', format='.1f')
        ]
    )

    # Combine layers for the second chart (Daily Trend)
    daily_trend_chart_linked_v4 = alt.layer(
        line_trend_filtered_v4, points_trend_filtered_v4
    ).properties(
        height=250, width=750, # Match width
        title='Daily Average Temperatures for Selected Month' # Simpler title
    )

    # Combine the two charts vertically
    linked_view_v4 = alt.vconcat(
        daily_cycle_chart_linked_v4,
        daily_trend_chart_linked_v4,
        spacing=20
    ).resolve_legend( # Resolve legends if needed
        color="independent", strokeWidth="independent"
    ).configure_axis(
        grid=True, gridColor='lightgray', labelFontSize=10, titleFontSize=12
    ).configure_title(
        fontSize=14, anchor='middle'
    ).configure_view(
        stroke=None
    ).interactive() # Make combined chart interactive (zoom/pan)


    # Save the chart specification
    try:
        linked_view_v4.save(v4_spec_path)
        print(f"  Saved V4 (Linked Cycles/Trend) spec to {v4_spec_path}")
    except Exception as e:
        print(f"  Could not save V4 spec: {e}")
else:
    print(f"  Skipping V4 spec generation: Data file(s) not found. Check {v4_cycles_data_path} and {v4_trend_data_path}")

print("(Success) Block 16: V4 spec generation complete.")


Block 16: Generating Altair spec for V4 (Linked Daily Cycles & Trend)...
  Saved V4 (Linked Cycles/Trend) spec to specs/cycles_trend_linked.json
(Success) Block 16: V4 spec generation complete.


In [98]:
# --- Block 17: Generate Altair Spec for V5 Time Series (Focus + Context) ---

print("\nBlock 17: Generating Altair spec for V5 Time Series (Focus + Context)...")

import os
import altair as alt
# Import pandas just in case it's needed for internal Altair operations
import pandas as pd

v5_ts_spec_path = 'specs/soil_precip_interactive_timeseries.json'
v5_ts_data_path = 'data/daily_soil_precip_timeseries.json'

if os.path.exists(v5_ts_data_path):
    # 1) Define the brush (interval) selection on the x‐axis only
    time_brush_v5 = alt.selection_interval(encodings=['x'], name='time_brush_v5')

    # 2) Base chart
    base_v5_ts = alt.Chart(alt.Data(url=v5_ts_data_path)).properties(width=700)

    # 3) Precipitation context (brush target)
    precip_context_v5 = base_v5_ts.mark_bar(color='steelblue', opacity=0.7).encode(
        x=alt.X(
            'day:T',
            title='Date (Brush to Select Range)',
            axis=alt.Axis(format='%b %d', grid=True)
        ),
        y=alt.Y(
            'precipitation:Q',
            title='Daily Precip (in)',
            axis=alt.Axis(titleColor='steelblue', titlePadding=10)
        ),
        tooltip=[
            alt.Tooltip('day:T',          format='%Y-%m-%d', title='Date'),
            alt.Tooltip('precipitation:Q', format='.2f',      title='Precip (in)')
        ]
    ).add_params( # Apply brush selection capability TO this chart
        time_brush_v5
    ).properties(
        height=80,
        title="Precipitation (Select Date Range Below)"
    )

    # 4) Soil moisture detail (filtered by brush)
    soil_detail_v5 = base_v5_ts.mark_line(
        point=True, color='saddlebrown', strokeWidth=2
    ).encode(
        x=alt.X(
            'day:T',
            title=None,
            axis=alt.Axis(labels=False, grid=True) # Keep grid for alignment
        ),
        y=alt.Y(
            'soil_moisture:Q',
            title='Soil Moisture (% Scaled)',
            axis=alt.Axis(titleColor='saddlebrown', titlePadding=10)
        ),
        tooltip=[
            alt.Tooltip('day:T',             format='%Y-%m-%d', title='Date'),
            alt.Tooltip('soil_moisture:Q',   format='.1f',       title='Soil Moisture (%)')
        ]
    ).transform_filter( # Filter this chart's data BY the brush selection
        time_brush_v5
    ).properties(
        height=300,
        title='Soil Moisture Response'
    )

    # 5) Combine WITHOUT enabling panning/zooming on the combined view
    viz1_interactive_v5 = alt.vconcat(
        soil_detail_v5,
        precip_context_v5,
        spacing=5
    ).resolve_scale(
        x='independent',
        y='independent'
    ).configure_axis(
        grid=True,
        gridColor='lightgray',
        labelFontSize=10,
        titleFontSize=12
    ).configure_title(
        fontSize=14,
        anchor='middle'
    ).configure_view(
        stroke=None
    )
    # <<<--- .interactive() call REMOVED here ---<<<

    # 6) Save the spec
    try:
        viz1_interactive_v5.save(v5_ts_spec_path)
        print(f"  Saved V5 (Soil/Precip Time Series - Static View / Brush Filter) spec to {v5_ts_spec_path}")
    except Exception as e:
        print(f"  Could not save V5 Time Series spec: {e}")

else:
    print(f"  Skipping V5 Time Series spec generation: Data file not found at {v5_ts_data_path}")

print("(Success) Block 17: V5 Time Series spec generation complete.")


Block 17: Generating Altair spec for V5 Time Series (Focus + Context)...
  Saved V5 (Soil/Precip Time Series - Static View / Brush Filter) spec to specs/soil_precip_interactive_timeseries.json
(Success) Block 17: V5 Time Series spec generation complete.


In [99]:
# --- Block 18: Generate Altair Spec for V5 Lag Correlation ---

print("\nBlock 18: Generating Altair spec for V5 Lag Correlation...")

v5_lag_spec_path = 'specs/soil_precip_lag_correlation.json' # Matches Block 11 save path
v5_lag_data_path = 'data/soil_precip_lag_correlation.json'

# Check if data file exists and load it to find peak for annotation
if os.path.exists(v5_lag_data_path):
    try:
        lag_corr_data = pd.read_json(v5_lag_data_path)
        if lag_corr_data.empty or 'correlation' not in lag_corr_data.columns:
            print("  Lag correlation data file is empty or missing 'correlation' column.")
            peak_lag_v5 = 0; peak_corr_v5 = 0; peak_text_v5 = 'No data'
        else:
            # Find peak correlation details for annotation
            peak_idx_v5 = lag_corr_data['correlation'].idxmax()
            peak_lag_v5 = lag_corr_data.loc[peak_idx_v5, 'lag']
            peak_corr_v5 = lag_corr_data.loc[peak_idx_v5, 'correlation']
            peak_text_v5 = f'Peak correlation at {int(peak_lag_v5)} day lag: {peak_corr_v5:.3f}'
            print(f"  Peak correlation calculated: {peak_text_v5}")

        # Base chart for the correlation plot
        base_corr_v5 = alt.Chart(alt.Data(url=v5_lag_data_path)).properties(
             width=500, height=300,
             title='Correlation Between Soil Moisture and Lagged Precipitation'
        )

        # Line and points representing the correlation values by lag
        line_corr_v5 = base_corr_v5.mark_line(point=True, color='#B5179E', strokeWidth=2, size=80).encode(
            x=alt.X('lag:O', title='Lag (days)', axis=alt.Axis(labelAngle=0)), # Ordinal axis for discrete lags
            y=alt.Y('correlation:Q', title='Correlation Coefficient', scale=alt.Scale(zero=True)),
            tooltip=[ alt.Tooltip('lag:O', title='Lag (days)'), alt.Tooltip('correlation:Q', format='.3f', title='Correlation') ]
        )

        # Horizontal rule at y=0
        zero_rule_v5 = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(color='black', strokeDash=[3,3], strokeWidth=1).encode(y='y')

        # Annotation Text displaying the peak correlation info
        # Create a DataFrame for the annotation text position
        min_corr = lag_corr_data['correlation'].min() if not lag_corr_data.empty else 0
        annotation_y_pos = min_corr - 0.05 # Position slightly below the min correlation
        annotation_df_v5 = pd.DataFrame({'lag': [peak_lag_v5], 'correlation': [annotation_y_pos], 'text': [peak_text_v5]})

        annotation_text_v5 = alt.Chart(annotation_df_v5).mark_text(
            align='center', fontSize=11, color='black', dy=0 # Adjust vertical position slightly if needed
        ).encode(
            x=alt.X('lag:O'), y=alt.Y('correlation:Q'), text='text:N'
        )

        # Layer the chart components
        viz2_correlation_v5 = alt.layer(
            line_corr_v5, zero_rule_v5, annotation_text_v5
        ).configure_axis(
            grid=True, gridColor='lightgray', labelFontSize=10, titleFontSize=12
        ).configure_title(
            fontSize=14, anchor='middle'
        ).configure_view(
            stroke=None
        ).interactive()

        # Save the chart specification
        try:
            viz2_correlation_v5.save(v5_lag_spec_path)
            print(f"  Saved V5 (Lag Correlation) spec to {v5_lag_spec_path}")
        except Exception as e:
            print(f"  Could not save V5 Lag Correlation spec: {e}")

    except Exception as read_err:
        print(f"  Error reading lag correlation data file {v5_lag_data_path}: {read_err}")
        print(f"  Skipping V5 Lag Correlation spec generation.")

else:
    print(f"  Skipping V5 Lag Correlation spec generation: Data file not found at {v5_lag_data_path}")

print("(Success) Block 18: V5 Lag Correlation spec generation complete.")


Block 18: Generating Altair spec for V5 Lag Correlation...
  Peak correlation calculated: Peak correlation at 3 day lag: 0.359
  Saved V5 (Lag Correlation) spec to specs/soil_precip_lag_correlation.json
(Success) Block 18: V5 Lag Correlation spec generation complete.


In [100]:
# --- Block 19: Generate Altair Spec for V6 (Temp vs Wind Speed Scatter Plot) ---

print("\nBlock 19: Generating Altair spec for V6 (Temp vs Wind Speed Scatter Plot)...")

v6_spec_path = 'specs/temp_wind_scatter.json'
v6_data_path = 'data/temp_wind_daily_march.json' # March data

# Check if data file exists and load for calculations
if os.path.exists(v6_data_path):
    slope_v6, intercept_v6, correlation_v6 = np.nan, np.nan, np.nan
    trend_text_v6 = "Trend: N/A"
    corr_text_v6 = "Corr: N/A"
    annotation_data_v6 = pd.DataFrame() # Initialize

    try:
        wind_temp_data_v6 = pd.read_json(v6_data_path)
        if not wind_temp_data_v6.empty and \
           'measurement_value_temp' in wind_temp_data_v6.columns and \
           'measurement_value_wind' in wind_temp_data_v6.columns and \
           pd.api.types.is_numeric_dtype(wind_temp_data_v6['measurement_value_temp']) and \
           pd.api.types.is_numeric_dtype(wind_temp_data_v6['measurement_value_wind']):

            # Ensure variance for regression/correlation
            if len(wind_temp_data_v6) > 1 and \
               wind_temp_data_v6['measurement_value_temp'].std() > 1e-6 and \
               wind_temp_data_v6['measurement_value_wind'].std() > 1e-6:

                z_v6 = np.polyfit(wind_temp_data_v6['measurement_value_temp'], wind_temp_data_v6['measurement_value_wind'], 1)
                slope_v6 = z_v6[0]
                intercept_v6 = z_v6[1]
                trend_text_v6 = f"Trend: y={slope_v6:.2f}x{intercept_v6:+.2f}" # Keep '+' sign

                correlation_v6 = wind_temp_data_v6['measurement_value_temp'].corr(wind_temp_data_v6['measurement_value_wind'])
                corr_text_v6 = f'Corr: {correlation_v6:.2f}'
                print(f"  Calculated V6: {trend_text_v6}, {corr_text_v6}")

                # Prepare annotation data - REVISED PLACEMENT
                # Find min/max after potential filtering (read from file)
                temp_min_v6, temp_max_v6 = wind_temp_data_v6['measurement_value_temp'].min(), wind_temp_data_v6['measurement_value_temp'].max()
                wind_min_v6, wind_max_v6 = wind_temp_data_v6['measurement_value_wind'].min(), wind_temp_data_v6['measurement_value_wind'].max()

                # Position annotations near top-left, relative to plot area
                # Use fixed values relative to data range for more control
                text_x_pos_v6 = temp_min_v6 + (temp_max_v6 - temp_min_v6) * 0.05 # 5% from left
                text_y_pos_corr_v6 = wind_max_v6 - (wind_max_v6 - wind_min_v6) * 0.05 # 5% from top
                text_y_pos_trend_v6 = wind_max_v6 - (wind_max_v6 - wind_min_v6) * 0.12 # 12% from top

                annotation_data_v6 = pd.DataFrame([
                    {'x': text_x_pos_v6, 'y': text_y_pos_corr_v6, 'text': corr_text_v6},
                    {'x': text_x_pos_v6, 'y': text_y_pos_trend_v6, 'text': trend_text_v6}
                ])

            else:
                 print("  Cannot calculate regression/correlation: insufficient data or variance.")
        else:
             print("  Cannot calculate regression/correlation: data is empty or columns invalid.")

    except Exception as read_err:
        print(f"  Error reading or processing V6 data file {v6_data_path}: {read_err}")

    # --- Define Chart Components ---

    # Base chart definition
    base_v6 = alt.Chart(alt.Data(url=v6_data_path)).encode(
        x=alt.X('measurement_value_temp:Q',
                title='Temperature (°C)',
                # Explicitly set scale domain based on data? Or let it auto-adjust? Auto is usually fine.
                # Example: scale=alt.Scale(domain=[temp_min_v6 - 2, temp_max_v6 + 2])
                scale=alt.Scale(zero=False)
               ),
        y=alt.Y('measurement_value_wind:Q',
                title='Wind Speed (m/s)',
                # Explicitly set scale domain?
                # Example: scale=alt.Scale(domain=[wind_min_v6 - 1, wind_max_v6 + 1])
                scale=alt.Scale(zero=False) # Let wind speed scale start near data min
               )
    )

    # Layer 1: Scatter points
    points_v6 = base_v6.mark_point(size=80, opacity=0.7, filled=True, color='darkgreen').encode(
        tooltip=[
            alt.Tooltip('day:T', format='%Y-%m-%d', title='Date'),
            alt.Tooltip('measurement_value_temp:Q', format='.1f', title='Avg Temp (°C)'),
            alt.Tooltip('measurement_value_wind:Q', format='.1f', title='Avg Wind (m/s)'),
            alt.Tooltip('date_label:N', title='Date Label')
        ]
    ).properties( # Move title to combined chart
         #title='Temperature vs. Wind Speed in Chicago (March 2017)'
    )

    # Layer 2: Regression line
    regression_line_v6 = alt.Chart().mark_line() # Initialize empty
    if pd.notna(slope_v6): # Only add if calculated
        regression_line_v6 = base_v6.mark_line(color="red", strokeDash=[3,3], strokeWidth=2).transform_regression(
            'measurement_value_temp', 'measurement_value_wind', method='linear'
        )

    # Layer 3: Annotation text
    # Use mark_text directly on the annotation data frame
    annotation_text_v6 = alt.Chart(annotation_data_v6).mark_text(
        align='left',
        baseline='top', # Align top of text to the y-value
        dx=5,  # Small horizontal offset from the left-aligned x-position
        dy=5,  # Small vertical offset downwards from the top-aligned y-position
        fontSize=10, # Slightly smaller font?
        color='black'
        ).encode(
        x='x:Q',
        y='y:Q',
        text='text:N'
    ) if not annotation_data_v6.empty else alt.Chart()

    # --- Combine Layers ---
    # Layer order: points -> line -> text annotations
    chart_layers_v6 = alt.layer(
        points_v6,
        regression_line_v6,
        annotation_text_v6,
        # No dummy legend needed if we don't represent trend in legend
    ).properties(
        width=700, height=450, # Adjust size if needed
        title='Temperature vs. Wind Speed in Chicago (March 2017)' # Set title on combined chart
    ).configure_axis( # Ensure only one y-axis is configured
        grid=True, gridColor='lightgray', labelFontSize=10, titleFontSize=12
    ).configure_title(
        fontSize=14, anchor='middle'
    ).configure_view(
        stroke=None # Remove outer border
    )

    # Enable interactivity
    chart_v6 = chart_layers_v6.interactive()

    # --- Save the chart specification ---
    try:
        chart_v6.save(v6_spec_path)
        print(f"  Saved V6 (Temp vs Wind Scatter - Updated) spec to {v6_spec_path}")
    except Exception as e:
        print(f"  Could not save V6 spec: {e}")

else:
    print(f"  Skipping V6 spec generation: Data file not found at {v6_data_path} or was empty.")

print("(Success) Block 19: V6 spec generation complete.")


Block 19: Generating Altair spec for V6 (Temp vs Wind Speed Scatter Plot)...
  Calculated V6: Trend: y=-0.09x+4.87, Corr: -0.38
  Saved V6 (Temp vs Wind Scatter - Updated) spec to specs/temp_wind_scatter.json
(Success) Block 19: V6 spec generation complete.


In [101]:
# --- End of Integrated Blocks ---
print("\n--- All Processing and Specification Generation Complete ---")

# Display final message about outputs
print("\nData files saved in the 'data/' directory.")
print("Altair JSON specifications saved in the 'specs/' directory.")
print("These JSON files can now be used with vegaEmbed in your HTML/JavaScript application.")


--- All Processing and Specification Generation Complete ---

Data files saved in the 'data/' directory.
Altair JSON specifications saved in the 'specs/' directory.
These JSON files can now be used with vegaEmbed in your HTML/JavaScript application.


1.) Temperature Evolution During Winter-to-Spring Transition

Daily temperature trend line with min/max range
Monthly violin plots showing distribution changes


2.) Spatial Temperature Variations

Monthly temperature maps showing geographic patterns
Temperature variability map highlighting areas with greatest fluctuations


3.) Environmental Measurement Relationships

Temperature-humidity correlation scatter plot with time progression
Combined time series of temperature, humidity, and precipitation


4.) Soil Moisture Response to Precipitation

Time series showing precipitation events and soil moisture response
Lag correlation analysis showing delayed response patterns


5.) Wind Patterns and Correlations

Wind rose diagrams for different Chicago locations
Temperature-wind speed relationship scatter plot


6.) Daily Cycles and Seasonal Changes

Monthly comparison of daily temperature cycles
Daily temperature range progression over the study period
Hourly temperature heatmap showing daily and seasonal patterns